In [107]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [108]:
# sensor and ground truth data 
gyro  = pd.read_csv("archive/RawGyro.csv")
accel = pd.read_csv("archive/RawAccel.csv")
baro  = pd.read_csv("archive/BarometricPressure.csv")
gps   = pd.read_csv("archive/OnboardGPS.csv")
truth = pd.read_csv("archive/OnboardPose.csv")

# data cleaning
gps = gps.drop(['Unnamed: 14', 'Unnamed: 15',
       'Unnamed: 16', 'Unnamed: 17'], axis=1)
# these columns were either empty or were all NaN's

### Compute accel, gyro, baro biases


We'll compute sensor biases by computing residuals with respect to `truth` which is the pose computed by PIXHAWK autopilot http://pixhawk.org/modules/pixhawk, comes with the dataset. We formulate: residual = true value - sensor read and bias = avg(residual)

### Frequencies

- Pose data has frequency 50 Hz, or updates every 0.02 seconds.
- Accel, Gyro, Baro have frequency 10 Hz, or update every 0.1 second
- GPS has frequency 30 Hz, or updates every 0.03 seconds.

Gyro, accel, baro freq vs. pose freq: 50 Hz vs. 10 Hz, so we'll adjust the freq difference by group averaging.
The data is extracted from Kaggle, The Zurich Urban Micro Aerial Vehicle Dataset. Details on the dataset can be found here: https://rpg.ifi.uzh.ch/docs/IJRR17_Majdik.pdf

In [109]:
# slice angular velocity data from ground truth
truth_omega = truth[[" Omega_x", " Omega_y", " Omega_z"]].to_numpy()
# prep to perform grouping on the slice to adjust frequency
truth_omega.shape

(135098, 3)

In [110]:
# trim the slice to fit the grouping reqs
n             = truth.shape[0] // 5
truth_trimmed = truth_omega[:n*5]
# get rid of last three samples
truth_trimmed.shape

(135095, 3)

In [111]:
# perform grouping and compute mean
gyro_truth = truth_trimmed.reshape(n, 5, 3).mean(axis=1)
gyro_truth.shape

(27019, 3)

In [112]:
gyro_measured = gyro[[" x", " y", " z"]].to_numpy()
gyro_measured.shape

(27050, 3)

In [113]:
# drop 27050-27019 rows from the end of gyro_measured to align shapes
gyro_measured = gyro_measured[:27019]
gyro_measured.shape

(27019, 3)

In [114]:
# gyro bias 
gyro_residuals = gyro_measured - gyro_truth
gyro_bias      = np.mean(gyro_residuals, axis=0)              # constant offset per axis
gyro_noise_cov = np.cov((gyro_residuals - gyro_bias).T)       # covariance matrix


truth_accel     = truth[[" Accel_x", " Accel_y", " Accel_z"]].to_numpy()
truth_trimmed   = truth_accel[:n*5]  # drop remaining rows

# reassign truth
truth_accel     = truth_trimmed.reshape(n, 5, 3).mean(axis=1)

# accel bias
accel_measured  = accel[[" x", " y", " z"]].to_numpy()
accel_measured  = accel_measured[:truth_accel.shape[0]]
accel_residuals = accel_measured - truth_accel
accel_bias      = np.mean(accel_residuals, axis=0)
accel_noise_cov = np.cov((accel_residuals - accel_bias).T)

In [115]:
truth_alt     = truth[" Altitude"].to_numpy()
truth_trimmed = truth_alt[:n*5]  # drop remaining rows

In [116]:
z_coord_t = truth_trimmed.reshape(n, 5).mean(axis=1)
truth_alt = np.column_stack((np.zeros_like(z_coord_t), np.zeros_like(z_coord_t), z_coord_t))

In [117]:
np.var((baro_residuals - baro_bias)[:,2])

0.045181890908440485

In [118]:
# reassign truth
baro_measured = baro[" Altitude"].to_numpy()
z_coord       = np.array(baro_measured[:truth_alt.shape[0]])
baro_measured = np.column_stack((np.zeros_like(z_coord), np.zeros_like(z_coord), z_coord))

# baro bias
baro_residuals = baro_measured - truth_alt
baro_bias      = np.mean(baro_residuals, axis = 0)
baro_noise_var = np.diag([0, 0, np.var((baro_residuals - baro_bias)[:,2])])

# we have obtained biases for each IMU sensor
baro_bias, accel_bias, gyro_bias

(array([ 0.        ,  0.        , -3.03693697]),
 array([  0.69995173,   0.21961004, -18.5292454 ]),
 array([-0.01736213, -0.02579525,  0.02788524]))

In [119]:
# we have also obtained the sensors' measurement noise covariance matrices
accel_noise_cov, gyro_noise_cov, baro_noise_var

(array([[1.07082667, 0.39578229, 0.12938442],
        [0.39578229, 0.26699387, 0.02161694],
        [0.12938442, 0.02161694, 0.46484711]]),
 array([[0.01002963, 0.00068271, 0.00038498],
        [0.00068271, 0.00845412, 0.00020865],
        [0.00038498, 0.00020865, 0.00897964]]),
 array([[0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.14959922]]))

Accelerometer's and Gyroscope's covariances make it into the process model. On this iteration of the analysis, I will leave generating the noise matrix for simplicity. However, we still need the GPS covariance matrices for position and velocity before we can run the filter. 

In [120]:
#  compute the correlation matrix for position

## convert ECEF (Earth centered, Earth fixed) to ENU (East, North, Up)

### following readme.txt: transform from 1e7 degrees to degrees

gps_lat = gps[' lat'].to_numpy() / 1e7
gps_lon = gps[' lon'].to_numpy() / 1e7
gps_alt = gps[' alt'].to_numpy() / 1000  # mm to m


R = 6371000 # Earth's radius in meters
init_lat = gps_lat[0]
init_lon = gps_lon[0]
init_alt = gps_alt[0]

### simplified ECEF to ENU conversion

gps_x_coord = (gps_lon - init_lon) * np.cos(np.deg2rad(init_lat)) * (np.pi/180) * R
gps_y_coord = (gps_lat - init_lat) * (np.pi/180) * R
gps_z_coord = (gps_alt - init_alt)


gps_pos = np.stack((gps_x_coord, gps_y_coord, gps_z_coord), axis=1)
# compute the correlation matrix for velocity


In [121]:
# load the data
groundtruth = pd.read_csv("archive/GroundTruthAGL.csv")
groundtruth_pos = groundtruth[[" x_gt", " y_gt", " z_gt"]].to_numpy()

In [122]:
gps.shape[0]/groundtruth.shape[0], gps.shape[0], groundtruth.shape[0]

(29.97378138847858, 81169, 2708)

In [123]:
# pack gps data into groups of 30
n = gps.shape[0]//30
gps_trimmed = gps_pos[:n*30]

In [124]:
gps_trimmed.shape

(81150, 3)

In [125]:
gps_pos = gps_trimmed.reshape(n, 30, 3).mean(axis=1)
groundtruth_pos = groundtruth_pos[0:gps_pos.shape[0]]
gps_pos.shape[0], groundtruth_pos.shape[0]

(2705, 2705)

In [126]:
# gps position bias 
gps_pos_residuals = gps_pos - groundtruth_pos
gps_pos_bias = np.mean(gps_pos_residuals, axis=0)              # constant offset per axis
gps_pos_noise_cov = np.cov((gps_pos_residuals - gps_pos_bias).T)   # covariance matrix

In [127]:
# GPS velocity covariance matrix
## Because of the absence of ground truth, we'll use isotropic velocity variation
gps_vel_std = gps[[" s_variance_m_s"]].to_numpy().mean()
gps_vel_noise_cov = np.diag([gps_vel_std**2, gps_vel_std**2, gps_vel_std**2])
gps_vel_noise_cov

array([[1.28992823, 0.        , 0.        ],
       [0.        , 1.28992823, 0.        ],
       [0.        , 0.        , 1.28992823]])

Two types of noises: process noise $Q$, measurment noise $R$. $Q$ tell us variations within our process given the non-deterministic nature of the environment and $R$ tells us the variations within our measurements.  

In [128]:
%store gps_vel_noise_cov
%store gps_pos_noise_cov
%store accel_noise_cov 
%store gyro_noise_cov
%store baro_noise_var
%store baro_bias
%store accel_bias
%store gyro_bias

Stored 'gps_vel_noise_cov' (ndarray)
Stored 'gps_pos_noise_cov' (ndarray)
Stored 'accel_noise_cov' (ndarray)
Stored 'gyro_noise_cov' (ndarray)
Stored 'baro_noise_var' (ndarray)
Stored 'baro_bias' (ndarray)
Stored 'accel_bias' (ndarray)
Stored 'gyro_bias' (ndarray)
